In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-olympics-in-tokyo/EntriesGender.xlsx
/kaggle/input/2021-olympics-in-tokyo/Teams.xlsx
/kaggle/input/2021-olympics-in-tokyo/Athletes.xlsx
/kaggle/input/2021-olympics-in-tokyo/Coaches.xlsx
/kaggle/input/2021-olympics-in-tokyo/Medals.xlsx


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=095989425f141fbb58675ef348f406b25b4cf9f8a06da09135255cf03be30eeb
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [4]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 11:33:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
os.listdir('/kaggle/working')

['.virtual_documents', '__notebook_source__.ipynb']

In [18]:
import pandas as pd

def excel_to_csv(name):
    pd.read_excel( '/kaggle/input/2021-olympics-in-tokyo/' + name + '.xlsx').to_csv(name + '.csv', sep=',', index=False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('Medals')
excel_to_csv('Teams')
excel_to_csv('EntriesGender')

/opt/conda/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [20]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')
entries_gender = pd.read_csv('EntriesGender.csv')

In [21]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [23]:
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [26]:
import plotly.express as px

fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], 
             color_discrete_sequence = ['Gold', 'Silver', 'Brown'], 
             title='Number of medals per country')
fig.show()

In [30]:
coaches.head(5)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [32]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [35]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from Coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(7)

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [37]:
fig = px.bar(pd_coaches_query, x='NOC', y='Count', color='Count', title='Number of Coaches from Each Country')
fig.show()

In [38]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [43]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show(10)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
+--------------------+-----------+-------------------+
only showing top 10 rows



In [45]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
count(*) as Count from athletes group by NOC, Discipline
order by NOC, Discipline''').show(20)

+-----------+-------------------+-----+
|        NOC|         Discipline|Count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [46]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
count(*) as Count from athletes group by NOC, Discipline
order by NOC, Discipline''').toPandas()

In [48]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of athletes from each country or discipline')
fig.show()

In [49]:
athletes['Discipline'].value_counts().head(10)

Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: Discipline, dtype: int64

In [50]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [51]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of athletes from each country or discipline')
fig.show()

In [52]:
entries_gender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [53]:
len(entries_gender)

46

In [54]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type': 'domain'}, {'type':'domain'}]]*23
specs

[[{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}]]

In [55]:
fig = make_subplots(rows=23, cols=2,
                   subplot_titles = entries_gender['Discipline'], specs=specs
                   )
sub_figs = []
for index, row in entries_gender.iterrows():
    sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'],
                                                       row['Male']])
    sub_figs.append(sub_fig)
    
k = 0
for i in range(1, 24):
    for j in range(1, 3):
        fig.add_trace(sub_figs[k], i, j)
        k += 1
        
fig.update_layout(showlegend=False, height=10000, width=800, title_text="Distribution of Gender amongst each game")
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo="label+value+percent")
fig = go.Figure(fig)
fig.show()
